<a href="https://colab.research.google.com/github/Gatwaza/Gatwaza/blob/main/TorchIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'human-detection-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F717483%2F3174587%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240318%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240318T092116Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D92b5e49ac5daedaea129777c54eeb56394aa2018d903b802e24be51f685d6ee4738e25c910b5b331bd115239539756f3f6789e2fb60140841d5aa02a2a148dab8a597f99551d4ac18aecfef72d6a53eed0a04e58bcb00d6e546a2ac313cf02037535960dbcfdf31a093f3ad2ed8a86411e0caba4989054ef882de9e769b1224d838eb9acdbf0f05c416e08635d6b6673e9d7936db1e62148747c00e6fdb97540276e0197a9e99bbad96c18d6fc66bd5a37ef728c1d2f9e00e589411a260f30e0fd6beb89140311a2ea11f79746118d10ecbc119629ff2164593d6d63858441ea556c6067e9b5780a9857decd3a2fb30ec4c5a1886793b75c3f4f1206571d3f4e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 272336530 bytes downloaded
Downloaded and uncompressed: human-detection-dataset
Data source import complete.


In [8]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from keras.applications import MobileNetV2 #Pre-trained deep learning model for image classification.
from tensorflow.keras.optimizers import Adam #an optimization algorithm for weights update
from tensorflow.keras.models import Sequential #represents a linear stack of layers.
from tensorflow.keras.utils import to_categorical #convert integer class labels into one-hot encoded vectors.
from sklearn.model_selection import train_test_split #split data into training and testing sets for model evaluation.
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout #connected layer, flatten input into 1D array, normalizes the activations of previous layer and applies dropout regularization to the inputs.
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img #image preprocessing

In [9]:
main_dir = "/kaggle/input/human-detection-dataset/human detection dataset"


In [10]:
# Load and preprocess your dataset
def read_and_preprocess_image(image_path):
    image = cv2.imread(image_path, 1)
    image = cv2.resize(image, (224, 224))  # Resize to the input size of MobileNetV2
    return image

In [11]:
data = []
labels = []

for class_name in os.listdir(main_dir):
    class_dir = os.path.join(main_dir, class_name)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        image = read_and_preprocess_image(image_path)
        data.append(image)
        labels.append(int(class_name))  # Assuming class names are 0 and 1

# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=2)

In [12]:
len(data[0])

224

In [17]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [18]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

736
185
736
185


In [19]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [20]:
# Model architecture
model = Sequential()
base_model = MobileNetV2(input_shape=(224, 224, 3))
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

14536120/14536120 [==============================] - 0s 0us/step


In [21]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
# Train the model with data augmentation
batch_size = 32
epochs = 20

history = model.fit(datagen.flow(np.array(X_train), y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) / batch_size,
                    epochs=epochs,
                    validation_data=(np.array(X_test), y_test))

Epoch 1/20
23/23 [==============================] - 155s 6s/step - loss: 0.4488 - accuracy: 0.8125 - val_loss: 0.7287 - val_accuracy: 0.5676
Epoch 2/20
23/23 [==============================] - 120s 5s/step - loss: 0.3904 - accuracy: 0.8370 - val_loss: 0.7404 - val_accuracy: 0.5676
Epoch 3/20
23/23 [==============================] - 121s 5s/step - loss: 0.3108 - accuracy: 0.8723 - val_loss: 0.7829 - val_accuracy: 0.5676
Epoch 4/20
23/23 [==============================] - 118s 5s/step - loss: 0.3244 - accuracy: 0.8560 - val_loss: 0.7472 - val_accuracy: 0.5676
Epoch 5/20
23/23 [==============================] - 126s 5s/step - loss: 0.3012 - accuracy: 0.8832 - val_loss: 0.9135 - val_accuracy: 0.5892
Epoch 6/20
23/23 [==============================] - 121s 5s/step - loss: 0.2443 - accuracy: 0.9226 - val_loss: 0.8972 - val_accuracy: 0.5676
Epoch 7/20
23/23 [==============================] - 126s 5s/step - loss: 0.2364 - accuracy: 0.9062 - val_loss: 1.0501 - val_accuracy: 0.5676
Epoch 8/20
23

In [24]:
# Import TensorFlow
import tensorflow as tf

# Save the trained model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('TorchIt.tflite', 'wb') as f:
    f.write(tflite_model)

In [37]:
import tensorflow as tf
# Load the TFLite model
tflite_model_path = 'TorchIt.tflite'
tflite_model = tf.lite.Interpreter(model_path=tflite_model_path)

# Convert TFLite to TensorFlow SavedModel
saved_model_path = 'TorchIt'
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.experimental_new_converter = True  # Use the new TFLite converter

# Convert and save the model
tf_saved_model = converter.convert()
with tf.io.gfile.GFile('TorchIt/saved_model.pb', 'wb') as f:
    f.write(tf_saved_model)


In [38]:
!ls


sample_data  TorchIt  TorchIt.tflite
